In [1]:
!pip install '../'

Processing /Users/colmb/Library/CloudStorage/OneDrive-Graphcore/code/debugging-tool
  Preparing metadata (setup.py) ... done
  Created wheel for nvis: filename=nvis-0.1-py3-none-any.whl size=35894 sha256=a6eb87d9260482258b83aba95e5cd2c11cf5082a44cf5d50618b22f2d12fead0
  Stored in directory: /Users/colmb/Library/Caches/pip/wheels/a5/df/dc/5259f835417f3a97f1653a062131bd69e67749ca4095ea03fc
Successfully built nvis
  Attempting uninstall: nvis
    Found existing installation: nvis 0.1
    Uninstalling nvis-0.1:
      Successfully uninstalled nvis-0.1

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:


# import pandas as pd
from nvis import vis
from nvis.log.common import read_pickle, TensorType
import re
import pandas as pd
df = read_pickle('../test-data/example-logs.pkl')

# vis.interactive(
#     f= vis.scalar_line,
#     width=1000,
#     df=df,
#     tt=TensorType.Activation,
#     layer='output',
#     col_wrap = 3,
#     scalar_metric=['rm2','min_abs']
# )



In [3]:
vis.interactive(
    f= vis.scalar_line,
    df=df,
    tt='Activation',
    layer='output',
    col_wrap = 3,
    scalar_metric=['rm2','min_abs']
)

Output(layout=Layout(width='100%'), outputs=({'output_type': 'display_data', 'data': {'text/plain': "HBox(chil…

Output(layout=Layout(overflow='scroll hidden', width='1500px'), outputs=({'output_type': 'display_data', 'data…

In [5]:
vis.interactive(
    width=1400,
    f= vis.scalar_global_heatmap,
    df=df,
    tt='Activation',
    scalar_metric='rm2',
    inc=50
)

Output(layout=Layout(width='100%'), outputs=({'output_type': 'display_data', 'data': {'text/plain': "HBox(chil…

Output(layout=Layout(overflow='scroll hidden', width='1400px'), outputs=({'output_type': 'display_data', 'data…

In [4]:
vis.interactive(
    f=vis.exp_hist,
    df=df,
    layer='output',
    tt='Activation',
    step=100,
    col_wrap =3

)

Output(layout=Layout(width='100%'), outputs=({'output_type': 'display_data', 'data': {'text/plain': "HBox(chil…

Output(layout=Layout(overflow='scroll hidden', width='1500px'), outputs=({'output_type': 'display_data', 'data…

kind <class 'str'>
tt <class 'str'>
step <class 'int'>
layer <class 'list'>
